In [2]:
!pip install transformers datasets


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00


In [2]:
import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# Load IMDB dataset
dataset = load_dataset("imdb")

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Convert labels to torch tensors for training
tokenized_datasets = tokenized_datasets.map(lambda examples: {'labels': examples['label']}, batched=True)
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
# Load pre-trained BERT model for binary classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Load pre-trained BERT model for binary classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Sample a smaller portion of the dataset (10% of the original size)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(int(0.1 * len(tokenized_datasets["train"]))))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(int(0.1 * len(tokenized_datasets["test"]))))

# Adjust training arguments
training_args = TrainingArguments(
    output_dir="./results",               # Output directory
    evaluation_strategy="epoch",          # Evaluate at the end of each epoch
    learning_rate=2e-5,                   # Learning rate
    per_device_train_batch_size=4,        # Reduced batch size for training
    per_device_eval_batch_size=4,         # Reduced batch size for evaluation
    num_train_epochs=5,                   # Increase number of epochs to 5
    weight_decay=0.01,                    # Weight decay for regularization
    logging_steps=100,                    # Log every 100 steps to track loss more frequently
)

# Initialize the Trainer with updated arguments and smaller dataset
trainer = Trainer(
    model=model,                          # The pre-trained BERT model
    args=training_args,                   # Training arguments
    train_dataset=small_train_dataset,    # Smaller training dataset
    eval_dataset=small_eval_dataset,      # Smaller evaluation dataset
)

# Fine-tune the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.377100,0.415675
2,0.208900,0.679991
3,0.090200,0.402396
4,0.057500,0.577086
5,0.009900,0.561561


TrainOutput(global_step=3125, training_loss=0.17795407788932324, metrics={'train_runtime': 1897.4613, 'train_samples_per_second': 6.588, 'train_steps_per_second': 1.647, 'total_flos': 3288888192000000.0, 'train_loss': 0.17795407788932324, 'epoch': 5.0})

In [5]:
results = trainer.evaluate()
print(f"Evaluation results: {results}")
model.save_pretrained("./fine_tuned_bert_binary")


Evaluation results: {'eval_loss': 0.5615608096122742, 'eval_runtime': 77.6508, 'eval_samples_per_second': 32.195, 'eval_steps_per_second': 8.049, 'epoch': 5.0}


In [3]:
# Install the datasets library if you haven't already
# !pip install datasets

from datasets import load_dataset


datasets = load_dataset("imdb")

# Now, print the dataset structure
print(datasets)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [ ]:
from transformers import BertTokenizer

raw_datasets = load_dataset("imdb")
# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply tokenization
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Use 15-20% of the dataset (for Task 3)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(int(0.15 * len(tokenized_datasets["train"]))))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(int(0.15 * len(tokenized_datasets["test"]))))

from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained BERT model for binary classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,                   # Increased learning rate for Task 3
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=7,                   # Increased number of epochs for Task 3
    weight_decay=0.01,
    logging_steps=100,
)

# Initialize the Trainer with updated arguments and dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,    # 15-20% of dataset for Task 3
    eval_dataset=small_eval_dataset,
)

# Fine-tune the model
trainer.train()



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
